In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import time
from time import sleep
from random import randint
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
def parse_html(html):
    data, item = pd.DataFrame(), {}
    soup = BeautifulSoup(html, 'lxml')
    for i, resto in enumerate(soup.find_all('div', class_='location-meta-block')):
        
        name = str(resto.find('div',class_='result-title'))
        item['name'] = BeautifulSoup(name,"lxml").get_text()     
        
        number_reviews = str(resto.find('a',class_='review_count'))
        item['number_reviews'] = BeautifulSoup(number_reviews,"lxml").get_text()
        
        address = str(resto.find('div',class_='address'))
        item['address'] = BeautifulSoup(address,"lxml").get_text()
        
        rating = resto.select('.rating-review-count .ui_bubble_rating')
        item['rating'] = rating[0]['alt']
        data[i] = pd.Series(item)
    return data.T

In [3]:
restaurants = pd.DataFrame()
driver = webdriver.Chrome()
driver.maximize_window()

for i in range(0,991,30): #991
    url = f'https://www.tripadvisor.es/Search?q=Barcelona&searchSessionId=51C8E5CE54DA772C3953CD9DA7D126D31646914148673ssid&searchNearby=false&sid=6CE36F5C961D4332A3DC16D3040AA0ED1646914152244&blockRedirect=true&ssrc=e&rf=36&o={i}'
    driver.get(url)
    x = randint(2,6)
    sleep(x)
    new_data = parse_html(driver.page_source)
    restaurants = pd.concat([restaurants, new_data], ignore_index=True)
    print(len(restaurants))

driver.close()
restaurants

30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020


,name,number_reviews,address,rating
0,Barcelona Tapas Puerto Vallarta,2.429 opiniones,"Calle Matamoros 906, Puerto Vallarta, MéxicoCa...","4,5 de 5 burbujas"
1,BelleBuon,2.957 opiniones,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5 de 5 burbujas
2,Fàbrica Moritz Barcelona,2.887 opiniones,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4 de 5 burbujas
3,Ciudad Condal,10.183 opiniones,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...","4,5 de 5 burbujas"
4,La Peninsular,2.875 opiniones,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...","4,5 de 5 burbujas"
...,...,...,...,...
1015,GOLIARD,320 opiniones,"Carrer PROGRES, 6, Barcelona, Cataluña, España...","4,5 de 5 burbujas"
1016,Locavore Barcelona,126 opiniones,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5 de 5 burbujas
1017,Cal Marius 449 #pastrami,249 opiniones,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4 de 5 burbujas
1018,The Box,1.085 opiniones,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...","4,5 de 5 burbujas"


In [4]:
restaurants['rating']=restaurants['rating'].str.replace(' de 5 burbujas', '')
restaurants['number_reviews']=restaurants['number_reviews'].str.replace(' opiniones', '')

In [5]:
restaurants['rating']=restaurants['rating'].str.replace(',', '.')
restaurants['number_reviews']=restaurants['number_reviews'].str.replace('.', '')
restaurants['rating'] = restaurants['rating'].astype(float)
restaurants['number_reviews'] = restaurants['number_reviews'].astype(int)

In [6]:
restaurants

,name,number_reviews,address,rating
0,Barcelona Tapas Puerto Vallarta,2429,"Calle Matamoros 906, Puerto Vallarta, MéxicoCa...",4.5
1,BelleBuon,2957,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5.0
2,Fàbrica Moritz Barcelona,2887,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4.0
3,Ciudad Condal,10183,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...",4.5
4,La Peninsular,2875,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...",4.5
...,...,...,...,...
1015,GOLIARD,320,"Carrer PROGRES, 6, Barcelona, Cataluña, España...",4.5
1016,Locavore Barcelona,126,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5.0
1017,Cal Marius 449 #pastrami,249,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4.0
1018,The Box,1085,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...",4.5


In [7]:
restaurants = restaurants.drop_duplicates()
restaurants

,name,number_reviews,address,rating
0,Barcelona Tapas Puerto Vallarta,2429,"Calle Matamoros 906, Puerto Vallarta, MéxicoCa...",4.5
1,BelleBuon,2957,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5.0
2,Fàbrica Moritz Barcelona,2887,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4.0
3,Ciudad Condal,10183,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...",4.5
4,La Peninsular,2875,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...",4.5
...,...,...,...,...
1015,GOLIARD,320,"Carrer PROGRES, 6, Barcelona, Cataluña, España...",4.5
1016,Locavore Barcelona,126,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5.0
1017,Cal Marius 449 #pastrami,249,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4.0
1018,The Box,1085,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...",4.5


In [8]:
rest=restaurants

In [9]:
rest=rest[rest['address'].str.contains('Barcelona')]
rest

,name,number_reviews,address,rating
1,BelleBuon,2957,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5.0
2,Fàbrica Moritz Barcelona,2887,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4.0
3,Ciudad Condal,10183,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...",4.5
4,La Peninsular,2875,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...",4.5
5,La Patsa Lab,1758,"Calle Casanova 94, Barcelona, Cataluña, España...",5.0
...,...,...,...,...
1015,GOLIARD,320,"Carrer PROGRES, 6, Barcelona, Cataluña, España...",4.5
1016,Locavore Barcelona,126,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5.0
1017,Cal Marius 449 #pastrami,249,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4.0
1018,The Box,1085,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...",4.5


In [10]:
rest.reset_index(drop=True, inplace=True)
rest

,name,number_reviews,address,rating
0,BelleBuon,2957,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5.0
1,Fàbrica Moritz Barcelona,2887,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4.0
2,Ciudad Condal,10183,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...",4.5
3,La Peninsular,2875,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...",4.5
4,La Patsa Lab,1758,"Calle Casanova 94, Barcelona, Cataluña, España...",5.0
...,...,...,...,...
984,GOLIARD,320,"Carrer PROGRES, 6, Barcelona, Cataluña, España...",4.5
985,Locavore Barcelona,126,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5.0
986,Cal Marius 449 #pastrami,249,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4.0
987,The Box,1085,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...",4.5


In [11]:
def extract_street (s):
    return s.split(',')[0]

rest['street']=rest['address'].apply(extract_street)

In [12]:
def extract_number (s):
    return s.split(',')[1]

rest['number']=rest['address'].apply(extract_number)

In [13]:
rest

,name,number_reviews,address,rating,street,number
0,BelleBuon,2957,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5.0,Travessera de Gràcia 441,Barcelona
1,Fàbrica Moritz Barcelona,2887,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4.0,Ronda Sant Antoni 41,Barcelona
2,Ciudad Condal,10183,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...",4.5,Rambla de Catalunya,18
3,La Peninsular,2875,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...",4.5,Carrer Del Mar,29
4,La Patsa Lab,1758,"Calle Casanova 94, Barcelona, Cataluña, España...",5.0,Calle Casanova 94,Barcelona
...,...,...,...,...,...,...
984,GOLIARD,320,"Carrer PROGRES, 6, Barcelona, Cataluña, España...",4.5,Carrer PROGRES,6
985,Locavore Barcelona,126,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5.0,Carrer D'aribau,151
986,Cal Marius 449 #pastrami,249,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4.0,Mallorca,449
987,The Box,1085,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...",4.5,Carrer D'en Gignas 30,Barcelona


In [14]:
rest['street'] = rest['street'] + rest['number']

In [15]:
rest

,name,number_reviews,address,rating,street,number
0,BelleBuon,2957,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5.0,Travessera de Gràcia 441 Barcelona,Barcelona
1,Fàbrica Moritz Barcelona,2887,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4.0,Ronda Sant Antoni 41 Barcelona,Barcelona
2,Ciudad Condal,10183,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...",4.5,Rambla de Catalunya 18,18
3,La Peninsular,2875,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...",4.5,Carrer Del Mar 29,29
4,La Patsa Lab,1758,"Calle Casanova 94, Barcelona, Cataluña, España...",5.0,Calle Casanova 94 Barcelona,Barcelona
...,...,...,...,...,...,...
984,GOLIARD,320,"Carrer PROGRES, 6, Barcelona, Cataluña, España...",4.5,Carrer PROGRES 6,6
985,Locavore Barcelona,126,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5.0,Carrer D'aribau 151,151
986,Cal Marius 449 #pastrami,249,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4.0,Mallorca 449,449
987,The Box,1085,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...",4.5,Carrer D'en Gignas 30 Barcelona,Barcelona


In [16]:
rest['street']=rest['street'].str.replace('Barcelona', '')

In [17]:
rest

,name,number_reviews,address,rating,street,number
0,BelleBuon,2957,"Travessera de Gràcia 441, Barcelona, Cataluña,...",5.0,Travessera de Gràcia 441,Barcelona
1,Fàbrica Moritz Barcelona,2887,"Ronda Sant Antoni 41, Barcelona, Cataluña, Esp...",4.0,Ronda Sant Antoni 41,Barcelona
2,Ciudad Condal,10183,"Rambla de Catalunya, 18, Barcelona, Cataluña, ...",4.5,Rambla de Catalunya 18,18
3,La Peninsular,2875,"Carrer Del Mar, 29, Barcelona, Cataluña, Españ...",4.5,Carrer Del Mar 29,29
4,La Patsa Lab,1758,"Calle Casanova 94, Barcelona, Cataluña, España...",5.0,Calle Casanova 94,Barcelona
...,...,...,...,...,...,...
984,GOLIARD,320,"Carrer PROGRES, 6, Barcelona, Cataluña, España...",4.5,Carrer PROGRES 6,6
985,Locavore Barcelona,126,"Carrer D'aribau, 151, Barcelona, Cataluña, Esp...",5.0,Carrer D'aribau 151,151
986,Cal Marius 449 #pastrami,249,"Mallorca, 449, Barcelona, Cataluña, EspañaMall...",4.0,Mallorca 449,449
987,The Box,1085,"Carrer D'en Gignas 30, Barcelona, Cataluña, Es...",4.5,Carrer D'en Gignas 30,Barcelona


In [18]:
rest.drop(['number','address'], axis = 1, inplace = True)
rest.rename(columns = {'street':'address'}, inplace = True)
rest['city']='Barcelona'
rest

,name,number_reviews,rating,address,city
0,BelleBuon,2957,5.0,Travessera de Gràcia 441,Barcelona
1,Fàbrica Moritz Barcelona,2887,4.0,Ronda Sant Antoni 41,Barcelona
2,Ciudad Condal,10183,4.5,Rambla de Catalunya 18,Barcelona
3,La Peninsular,2875,4.5,Carrer Del Mar 29,Barcelona
4,La Patsa Lab,1758,5.0,Calle Casanova 94,Barcelona
...,...,...,...,...,...
984,GOLIARD,320,4.5,Carrer PROGRES 6,Barcelona
985,Locavore Barcelona,126,5.0,Carrer D'aribau 151,Barcelona
986,Cal Marius 449 #pastrami,249,4.0,Mallorca 449,Barcelona
987,The Box,1085,4.5,Carrer D'en Gignas 30,Barcelona


In [19]:
rest['address']=rest['address'].str.replace('C/', 'Carrer ')
rest['address']=rest['address'].str.replace('c/', 'Carrer ')
rest['address']=rest['address'].str.replace('Calle', 'Carrer ')
rest['address']=rest['address'].str.replace('calle', 'Carrer ')
rest['address']=rest['address'].str.replace('74b', '74')
rest['address']=rest['address'].str.replace('Carreer','Carrer ')
rest['address']=rest['address'].str.replace('Christina ','Cristina  ')
rest['address']=rest['address'].str.replace('[^\w\s]', ' ')

In [20]:
# for i in rest['address']:
#     if i in 're.sub('[^A-Za-z0-9]+', '', mystring)':
#         rest['address']=rest['address'].str.replace(i,'', inplace=True)

In [21]:
# error_check=rest['address'].str.contains('74b')

In [22]:
# rest.iloc[[11]]

In [23]:
rest

,name,number_reviews,rating,address,city
0,BelleBuon,2957,5.0,Travessera de Gràcia 441,Barcelona
1,Fàbrica Moritz Barcelona,2887,4.0,Ronda Sant Antoni 41,Barcelona
2,Ciudad Condal,10183,4.5,Rambla de Catalunya 18,Barcelona
3,La Peninsular,2875,4.5,Carrer Del Mar 29,Barcelona
4,La Patsa Lab,1758,5.0,Carrer Casanova 94,Barcelona
...,...,...,...,...,...
984,GOLIARD,320,4.5,Carrer PROGRES 6,Barcelona
985,Locavore Barcelona,126,5.0,Carrer D aribau 151,Barcelona
986,Cal Marius 449 #pastrami,249,4.0,Mallorca 449,Barcelona
987,The Box,1085,4.5,Carrer D en Gignas 30,Barcelona


In [24]:
rest['address']=rest['address']+' '+rest['city']

In [25]:
from geopy import Nominatim
from geopy.geocoders import Nominatim
import time

In [27]:
%%timeit
# from geopy.extra.rate_limiter import RateLimiter

#Creating an instance of Nominatim Class
geolocator = Nominatim(timeout=10, user_agent="sergiguasch83@gmail.com")

geocode = geolocator.geocode
 
#Applying the method to pandas DataFrame
rest['location'] = rest['address'].apply(geocode)
rest['lat'] = rest['location'].apply(lambda x: x.latitude if x else None)
rest['long'] = rest['location'].apply(lambda x: x.longitude if x else None)

In [28]:
# geolocator = Nominatim(timeout=10, user_agent = "SergiGuasch83@gmail.com")
# lat = []
# long = []
# index = -1

# for i in rest['address']:
#     location = geolocator.geocode(i, timeout=10)
#     index+=1
#     print(index)
#     if location is None:
#         lat.append(None)
#         long.append(None)
#     else:
#         latitude = location.latitude
#         longitude = location.longitude  
    
#     lat.append(latitude)
#     long.append(longitude)

In [29]:
# len(lat), len(long)

In [30]:
# rest['lat'] = pd.DataFrame(lat,columns=['lat'])
# rest['long'] = pd.DataFrame(long,columns=['long'])

In [31]:
pd.set_option('display.max_rows', None)

In [32]:
rest

,name,number_reviews,rating,address,city,location,lat,long
0,BelleBuon,2957,5.0,Travessera de Gràcia 441 Barcelona,Barcelona,"(441, Travessera de Gràcia, el Camp d'en Grass...",41.411243,2.172947
1,Fàbrica Moritz Barcelona,2887,4.0,Ronda Sant Antoni 41 Barcelona,Barcelona,"(Fábrica Moritz Barcelona, 41, Ronda de Sant A...",41.382567,2.163397
2,Ciudad Condal,10183,4.5,Rambla de Catalunya 18 Barcelona,Barcelona,"(Ciudad Condal, 18, Rambla de Catalunya, Eixam...",41.388800,2.166890
3,La Peninsular,2875,4.5,Carrer Del Mar 29 Barcelona,Barcelona,"(Bodega La Peninsular, 29, Carrer del Mar, la ...",41.380359,2.187872
4,La Patsa Lab,1758,5.0,Carrer Casanova 94 Barcelona,Barcelona,"(La Patsa Lab, 94, Carrer de Casanova, l'Antig...",41.386981,2.156921
5,Bella Italia,1384,5.0,Carrer de Roger de Lluria 87 Barcelona,Barcelona,"(El teu lloc, 87, Carrer de Roger de Llúria, l...",41.395029,2.166052
6,Thai Barcelona | Royal Cuisine,1654,4.0,Diputació 273 Barcelona,Barcelona,"(Diputació, Puigdàlber, Alt Penedès, Barcelona...",41.403974,1.699434
7,Ramen-ya Hiro,1212,4.5,Carrer Girona 164 Barcelona,Barcelona,"(Carrer Girona, Vilalba Sasserra, Vallès Orien...",41.653875,2.444301
8,Le Romane,2205,5.0,Carrer Muntaner 103 Barcelona,Barcelona,"(Le Romane, 103, Carrer de Muntaner, l'Antiga ...",41.388797,2.156331
9,Cervecería Catalana,12767,4.5,Carrer de Mallorca 236 Barcelona,Barcelona,"(Cervecería Catalana, 236, Carrer de Mallorca,...",41.392403,2.160901


In [33]:
rest.drop(['location'], axis = 1, inplace = True)

In [51]:
rest.to_csv('restaurants.csv',encoding='utf-8-sig')

In [1]:
# Load all importance packages
import geopandas
import numpy as np
import pandas as pd
from shapely.geometry import Point
import missingno as msn
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
gdf = geopandas.GeoDataFrame(rest, geometry=geopandas.points_from_xy(rest['long'], rest['lat']))

NameError: name 'rest' is not defined

In [ ]:
gdf

In [ ]:
type(gdf)

In [ ]:
type(gdf.geometry)

In [ ]:
type(gdf.geometry[0])

In [ ]:
gdf.plot(figsize=(30,20), color='#3B3C6E')

In [ ]:
import os
import folium

In [ ]:
gdf.crs = "EPSG:4326"
gdf = gdf.to_crs(epsg='4326')
ax = gdf.plot(cmap="viridis")

In [ ]:
# plot map
barcelona_map = folium.Map([41.3947,2.1557], zoom_start=12.4, tiles='cartodbpositron')
folium.GeoJson(gdf).add_to(barcelona_map)
barcelona_map

In [ ]:
rest

In [ ]:
rest.sort_values(by='number_reviews', ascending=False)

In [ ]:
rest.sort_values(by='lat', ascending=False)

In [ ]:
rest.sort_values(by='long', ascending=False)

In [ ]:
rest_nulls = rest[rest.isna().any(axis=1)]

In [ ]:
rest_nulls

In [ ]:
rest_nulls['lat'].isna().sum()